# Air Quality Data EDA - Multan AQI Features

This notebook analyzes the engineered air quality and weather data from Hopsworks feature store.

## Dataset Overview
- **Source**: Hopsworks Feature Store (multan_aqi_features)
- **Records**: 538 observations 
- **Features**: 127 engineered features
- **Time Range**: June 16, 2025 - July 8, 2025

## Modeling Approach
- **🎯 Goal**: Accurate US AQI prediction for Multan
- **🔧 Method**: Train ML model to predict PM2.5 & PM10 → Calculate AQI via EPA formula
- **📊 ML Targets**: pm2_5, pm10 concentrations (µg/m³)
- **✅ Success Metric**: How well calculated AQI matches actual AQI values

## Feature Categories
1. **Raw Air Quality**: pm2_5, pm10, co, no2, so2, o3, nh3
2. **AQI Calculations**: pm2_5_aqi, pm10_aqi, us_aqi, openweather_aqi
3. **Weather Data**: temperature, humidity, pressure, wind_speed, wind_direction
4. **Time Features**: Cyclical encodings (hour, day, month, etc.)
5. **Lag Features**: 1h-72h historical values
6. **Rolling Statistics**: 3h-24h windows (mean, std, min, max)
7. **Engineered Features**: Interactions, squared terms, categorical flags


# Air Quality Data EDA

This notebook analyzes the engineered air quality and weather data from Hopsworks feature store that will be used for modeling.

**EDA Focus**: Understanding relationships that help predict PM2.5 and PM10 concentrations accurately, which leads to better AQI predictions.

## 1. Data Overview
Loading and examining the basic structure of our modeling dataset from Hopsworks.


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hopsworks
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import configuration
from config import HOPSWORKS_CONFIG

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
# Connect to Hopsworks and load data
print("Connecting to Hopsworks...")
project = hopsworks.login(api_key_value=HOPSWORKS_CONFIG["api_key"], project=HOPSWORKS_CONFIG["project_name"])
fs = project.get_feature_store()

print("Loading feature group data...")
fg = fs.get_feature_group(HOPSWORKS_CONFIG["feature_group_name"], version=1)
df = fg.read()

print(f"Successfully loaded {len(df)} records from Hopsworks")
print(f"Date range: {df['time'].min()} to {df['time'].max()}")


In [ ]:
# Fix column references and prepare data
# The actual timestamp column is called 'time' not 'timestamp'
print("Data preparation and column check...")
print(f"Time column: {'time' if 'time' in df.columns else 'timestamp not found'}")
print(f"Date range: {df['time'].min()} to {df['time'].max()}")

# Ensure time is datetime
if df['time'].dtype == 'object':
    df['time'] = pd.to_datetime(df['time'])

# Sort by time
df = df.sort_values('time').reset_index(drop=True)
print("✓ Data sorted by time")



In [ ]:
# Basic dataset information
print("=" * 50)
print("DATASET OVERVIEW")
print("=" * 50)
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"Duration: {(df['time'].max() - df['time'].min()).days} days")
print()

print("COLUMNS:")
for i, col in enumerate(df.columns):
    print(f"{i+1:2d}. {col:<25} {str(df[col].dtype):<15}")

print()
print("FEATURE TYPES:")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'time' in numeric_cols:
    numeric_cols.remove('time')
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features ({len(numeric_cols)}): {numeric_cols}")
print(f"Datetime features ({len(datetime_cols)}): {datetime_cols}")
print(f"Categorical features ({len(categorical_cols)}): {categorical_cols}")


In [ ]:
# Display first and last few records
print("=" * 50)
print("SAMPLE DATA")
print("=" * 50)
print("\nFirst 5 records:")
display(df.head())

print("\nLast 5 records:")
display(df.tail())

print("\nRandom 5 records:")
display(df.sample(5))


In [ ]:
# Summary statistics for numeric features
print("=" * 50)
print("SUMMARY STATISTICS")
print("=" * 50)
display(df.describe())

print("\nAIR QUALITY FEATURES SUMMARY:")
aqi_features = ['pm2_5', 'pm10', 'co', 'no2', 'so2', 'o3', 'us_aqi', 'pm2_5_aqi', 'pm10_aqi']
aqi_present = [col for col in aqi_features if col in df.columns]
if aqi_present:
    display(df[aqi_present].describe())

print("\nWEATHER FEATURES SUMMARY:")
weather_features = ['temperature', 'feels_like', 'humidity', 'pressure', 'visibility', 'wind_speed', 'wind_direction', 'cloud_cover']
weather_present = [col for col in weather_features if col in df.columns]
if weather_present:
    display(df[weather_present].describe())


## 2. Time Series Analysis

Analyzing temporal patterns in PM concentrations (our prediction targets) and derived AQI values.

In [ ]:
# Time series plots for key air quality metrics
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# PM2.5 over time
axes[0].plot(df['time'], df['pm2_5'], alpha=0.7, color='red')
axes[0].set_title('PM2.5 Concentration Over Time')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].grid(True, alpha=0.3)

# PM10 over time  
axes[1].plot(df['time'], df['pm10'], alpha=0.7, color='orange')
axes[1].set_title('PM10 Concentration Over Time')
axes[1].set_ylabel('PM10 (µg/m³)')
axes[1].grid(True, alpha=0.3)

# US AQI over time
axes[2].plot(df['time'], df['us_aqi'], alpha=0.7, color='purple')
axes[2].set_title('US AQI Over Time')
axes[2].set_ylabel('US AQI')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

# Add AQI category colors as background
aqi_levels = [
    (0, 50, 'green', 'Good'),
    (51, 100, 'yellow', 'Moderate'), 
    (101, 150, 'orange', 'Unhealthy for Sensitive'),
    (151, 200, 'red', 'Unhealthy'),
    (201, 300, 'purple', 'Very Unhealthy'),
    (301, 500, 'maroon', 'Hazardous')
]

for min_val, max_val, color, label in aqi_levels:
    axes[2].axhspan(min_val, max_val, alpha=0.1, color=color, label=label)

plt.tight_layout()
plt.show()

# Time series summary - ML TARGETS and GOAL METRIC
print(f"Time Series Summary (ML Targets + Goal Metric):")
print(f"PM2.5 (ML Target): {df['pm2_5'].min():.1f} - {df['pm2_5'].max():.1f} µg/m³")
print(f"PM10 (ML Target):  {df['pm10'].min():.1f} - {df['pm10'].max():.1f} µg/m³") 
print(f"US AQI (Goal Metric): {df['us_aqi'].min():.1f} - {df['us_aqi'].max():.1f}")
print(f"\nModeling Approach: Predict PM concentrations → Calculate AQI → Evaluate AQI accuracy")


## 3. Feature Analysis and Correlations

Examining relationships between air quality and weather features.

In [ ]:
# Correlation analysis - ML targets + environmental predictors
key_features = [
    'pm2_5', 'pm10',  # ML targets
    'temperature', 'humidity', 'pressure', 'wind_speed',  # Weather predictors
    'carbon_monoxide', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide'  # Pollutant predictors
]
# Note: Excluding us_aqi since it's derived from PM targets

# Filter features that exist in our dataset
available_features = [col for col in key_features if col in df.columns]
print(f"Analyzing correlations for {len(available_features)} key features:")
print(available_features)

# Calculate correlation matrix
corr_matrix = df[available_features].corr()

# Create correlation heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Matrix (Key Variables)')
plt.tight_layout()
plt.show()

# Focus on ML TARGETS (PM concentrations) - what we need to predict well
print("\nStrongest correlations with PM2.5 (ML Target):")
pm25_corr = corr_matrix['pm2_5'].abs().sort_values(ascending=False)
for feature, corr in pm25_corr.head(8).items():
    if feature != 'pm2_5':
        print(f"  {feature:<20}: {corr:.3f}")

print("\nStrongest correlations with PM10 (ML Target):")
pm10_corr = corr_matrix['pm10'].abs().sort_values(ascending=False)
for feature, corr in pm10_corr.head(8).items():
    if feature != 'pm10':
        print(f"  {feature:<20}: {corr:.3f}")

print("\n🎯 CORRELATION FOCUS:")
print("Understanding which environmental factors help predict PM concentrations accurately")
print("Better PM predictions → More accurate AQI calculations")


In [ ]:
# Focus on ML TARGETS (PM concentrations) - what affects our predictions
print("="*60)
print("ML TARGET CORRELATION ANALYSIS")
print("="*60)

target_features = ['pm2_5', 'pm10']
predictor_features = ['temperature', 'humidity', 'pressure', 'wind_speed', 
                     'carbon_monoxide', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide']

available_predictors = [col for col in predictor_features if col in df.columns]

for target in target_features:
    if target in df.columns:
        print(f"\nStrongest correlations with {target.upper()} (Target Variable):")
        target_corr = df[[target] + available_predictors].corr()[target].abs().sort_values(ascending=False)
        for feature, corr in target_corr.head(6).items():
            if feature != target:
                print(f"  {feature:<20}: {corr:.3f}")

print(f"\n[EDA Focus: Understanding what predicts PM concentrations well]")
print(f"[Goal: Better PM predictions → More accurate AQI calculations]")


## 4. AQI Dominance Analysis

**Key Question**: Which pollutants actually drive AQI values? 

Since EPA AQI = MAX(individual pollutant AQIs), we need to check if other criteria pollutants sometimes create higher AQI than PM2.5/PM10. This validates our modeling approach of using only PM concentrations.

**EPA Criteria Pollutants Analyzed**: PM2.5, PM10, O3, CO, NO2, SO2  
*(Only these 6 pollutants have official EPA AQI breakpoints and affect AQI calculations)*


In [ ]:
# EPA AQI Calculation Functions for All Pollutants
def calculate_aqi_from_concentration(concentration, breakpoints):
    """Calculate AQI from pollutant concentration using EPA breakpoints"""
    if pd.isna(concentration) or concentration < 0:
        return 0
    
    for i, (c_low, c_high, aqi_low, aqi_high) in enumerate(breakpoints):
        if c_low <= concentration <= c_high:
            # Linear interpolation within the bracket
            aqi = ((aqi_high - aqi_low) / (c_high - c_low)) * (concentration - c_low) + aqi_low
            return round(aqi)
    
    # If concentration exceeds highest breakpoint, use hazardous level
    return 500

# EPA AQI Breakpoints (concentration ranges and corresponding AQI ranges)
EPA_BREAKPOINTS = {
    'pm2_5': [  # µg/m³, 24-hour average
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.4, 301, 500)
    ],
    'pm10': [  # µg/m³, 24-hour average  
        (0, 54, 0, 50),
        (55, 154, 51, 100),
        (155, 254, 101, 150),
        (255, 354, 151, 200),
        (355, 424, 201, 300),
        (425, 604, 301, 500)
    ],
    'ozone': [  # ppb, 8-hour average (converting from µg/m³ if needed)
        (0, 54, 0, 50),
        (55, 70, 51, 100),
        (71, 85, 101, 150),
        (86, 105, 151, 200),
        (106, 200, 201, 300)
    ],
    'carbon_monoxide': [  # ppm, 8-hour average
        (0.0, 4.4, 0, 50),
        (4.5, 9.4, 51, 100),
        (9.5, 12.4, 101, 150),
        (12.5, 15.4, 151, 200),
        (15.5, 30.4, 201, 300),
        (30.5, 50.4, 301, 500)
    ],
    'nitrogen_dioxide': [  # ppb, 1-hour average
        (0, 53, 0, 50),
        (54, 100, 51, 100),
        (101, 360, 101, 150),
        (361, 649, 151, 200),
        (650, 1249, 201, 300),
        (1250, 2049, 301, 500)
    ],
    'sulphur_dioxide': [  # ppb, 1-hour average
        (0, 35, 0, 50),
        (36, 75, 51, 100),
        (76, 185, 101, 150),
        (186, 304, 151, 200),
        (305, 604, 201, 300),
        (605, 1004, 301, 500)
    ]
}

print("EPA AQI Calculation Functions Loaded")
print(f"EPA Criteria Pollutants (official breakpoints): {list(EPA_BREAKPOINTS.keys())}")

# Check which EPA criteria pollutants we have in our data
available_pollutants = []
for pollutant in EPA_BREAKPOINTS.keys():
    if pollutant in df.columns:
        available_pollutants.append(pollutant)
        
print(f"\nEPA criteria pollutants in our dataset: {available_pollutants}")
print(f"Missing from dataset: {[p for p in EPA_BREAKPOINTS.keys() if p not in df.columns]}")
print(f"\nAnalyzing {len(available_pollutants)} pollutants that can affect AQI calculations")


In [ ]:
# Calculate individual AQI for each available pollutant
aqi_results = df[['time', 'us_aqi']].copy()

# Unit conversions (if needed)
df_calc = df.copy()

# Convert units for certain pollutants if needed
# Ozone: µg/m³ to ppb (approximate: µg/m³ * 0.5 ≈ ppb at standard conditions)
if 'ozone' in df_calc.columns:
    df_calc['ozone_ppb'] = df_calc['ozone'] * 0.5  # Rough conversion
    
# CO might need conversion from µg/m³ to ppm
if 'carbon_monoxide' in df_calc.columns:
    df_calc['co_ppm'] = df_calc['carbon_monoxide'] * 0.000873  # Rough conversion

print("Calculating individual AQI for each pollutant...")

# Calculate AQI for each pollutant
for pollutant in available_pollutants:
    col_name = f'{pollutant}_individual_aqi'
    
    if pollutant == 'ozone' and 'ozone_ppb' in df_calc.columns:
        concentrations = df_calc['ozone_ppb']
    elif pollutant == 'carbon_monoxide' and 'co_ppm' in df_calc.columns:
        concentrations = df_calc['co_ppm']
    else:
        concentrations = df_calc[pollutant]
    
    aqi_results[col_name] = concentrations.apply(
        lambda x: calculate_aqi_from_concentration(x, EPA_BREAKPOINTS[pollutant])
    )
    
    print(f"✓ {pollutant}: {aqi_results[col_name].min():.0f} - {aqi_results[col_name].max():.0f} AQI")

# Find controlling pollutant (max AQI) for each timestamp
individual_aqi_cols = [col for col in aqi_results.columns if 'individual_aqi' in col]
aqi_results['calculated_max_aqi'] = aqi_results[individual_aqi_cols].max(axis=1)
aqi_results['controlling_pollutant'] = aqi_results[individual_aqi_cols].idxmax(axis=1)

# Clean up pollutant names
aqi_results['controlling_pollutant'] = aqi_results['controlling_pollutant'].str.replace('_individual_aqi', '')

print(f"\nCalculated AQI range: {aqi_results['calculated_max_aqi'].min():.0f} - {aqi_results['calculated_max_aqi'].max():.0f}")
print(f"Current US AQI range: {aqi_results['us_aqi'].min():.0f} - {aqi_results['us_aqi'].max():.0f}")
print(f"Difference: {(aqi_results['calculated_max_aqi'] - aqi_results['us_aqi']).describe()}")


In [ ]:
# VISUALIZATION 1: Pollutant Dominance Analysis
print("="*60)
print("POLLUTANT DOMINANCE ANALYSIS")
print("="*60)

# Count which pollutant controls AQI most often
dominance_counts = aqi_results['controlling_pollutant'].value_counts()
dominance_pct = (dominance_counts / len(aqi_results) * 100).round(1)

print("Controlling Pollutant Frequency:")
for pollutant, count in dominance_counts.items():
    pct = dominance_pct[pollutant]
    print(f"  {pollutant:<15}: {count:3d} times ({pct:5.1f}%)")

# Pie chart of dominance
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.pie(dominance_counts.values, labels=dominance_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Which Pollutant Controls AQI?')

# Bar chart for clearer comparison
plt.subplot(1, 2, 2)
bars = plt.bar(range(len(dominance_counts)), dominance_counts.values, 
               color=['red' if x in ['pm2_5', 'pm10'] else 'lightcoral' for x in dominance_counts.index])
plt.xticks(range(len(dominance_counts)), [p.replace('_', ' ').title() for p in dominance_counts.index], rotation=45)
plt.ylabel('Number of Hours')
plt.title('Controlling Pollutant Frequency')

# Highlight PM2.5 and PM10
for i, (pollutant, count) in enumerate(dominance_counts.items()):
    color = 'white' if pollutant in ['pm2_5', 'pm10'] else 'black'
    plt.text(i, count + 5, f'{count}', ha='center', va='bottom', fontweight='bold', color=color)

plt.tight_layout()
plt.show()

# Analysis of PM dominance
pm_dominance = dominance_counts.get('pm2_5', 0) + dominance_counts.get('pm10', 0)
pm_percentage = (pm_dominance / len(aqi_results) * 100)

print(f"\n🎯 KEY FINDING:")
print(f"PM2.5 + PM10 control AQI {pm_dominance}/{len(aqi_results)} times ({pm_percentage:.1f}%)")
print(f"Other pollutants control AQI {len(aqi_results) - pm_dominance}/{len(aqi_results)} times ({100-pm_percentage:.1f}%)")

if pm_percentage >= 80:
    print("✅ Current PM-only approach captures most AQI variations")
else:
    print("⚠️  Consider including other pollutants in modeling")


In [ ]:
# VISUALIZATION 2: Concentration vs AQI Relationships
print("\n" + "="*60)
print("CONCENTRATION vs AQI ANALYSIS")
print("="*60)

# Create scatter plots for EPA criteria pollutants only
num_plots = len(available_pollutants)
cols = 3
rows = (num_plots + cols - 1) // cols  # Ceiling division

fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
if rows == 1:
    axes = [axes]  # Make it iterable
axes = axes.flatten()

for i, pollutant in enumerate(available_pollutants):
        
    ax = axes[i]
    
    # Get concentrations and individual AQI
    if pollutant == 'ozone' and 'ozone_ppb' in df_calc.columns:
        conc = df_calc['ozone_ppb']
        unit = 'ppb'
    elif pollutant == 'carbon_monoxide' and 'co_ppm' in df_calc.columns:
        conc = df_calc['co_ppm']
        unit = 'ppm'
    else:
        conc = df_calc[pollutant]
        unit = 'µg/m³'
    
    # All pollutants here are EPA criteria pollutants with AQI calculations
    individual_aqi = aqi_results[f'{pollutant}_individual_aqi']
    
    # Scatter plot with AQI color coding
    scatter = ax.scatter(conc, individual_aqi, alpha=0.6, s=20, 
                        c=individual_aqi, cmap='RdYlGn_r', vmin=0, vmax=150)
    
    ax.set_xlabel(f'{pollutant.replace("_", " ").title()} ({unit})')
    ax.set_ylabel('Individual AQI')
    ax.set_title(f'{pollutant.replace("_", " ").title()} → AQI')
    ax.grid(True, alpha=0.3)
    
    # Add AQI level lines
    ax.axhline(y=50, color='green', linestyle='--', alpha=0.5, linewidth=1)
    ax.axhline(y=100, color='yellow', linestyle='--', alpha=0.5, linewidth=1)
    ax.axhline(y=150, color='orange', linestyle='--', alpha=0.5, linewidth=1)

# Remove empty subplots
for i in range(len(available_pollutants), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Summary statistics for EPA criteria pollutants
print("\nIndividual AQI Statistics (EPA Criteria Pollutants):")
for pollutant in available_pollutants:
    aqi_col = f'{pollutant}_individual_aqi'
    stats = aqi_results[aqi_col].describe()
    print(f"{pollutant:<15}: Mean={stats['mean']:5.1f}, Max={stats['max']:5.1f}, >100: {(aqi_results[aqi_col] > 100).sum():3d} times")


In [ ]:
# VISUALIZATION 3A: Individual AQI Contributions vs Maximum AQI
print("\n" + "="*60)
print("INDIVIDUAL POLLUTANT AQI CONTRIBUTIONS")
print("="*60)

# Time series showing all individual AQIs vs the maximum
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Plot 1: All individual AQIs over time
time_vals = aqi_results['time']
max_aqi_vals = aqi_results['calculated_max_aqi']

# Plot each individual AQI
colors = ['red', 'darkred', 'blue', 'purple', 'orange', 'green']
for i, pollutant in enumerate(available_pollutants):
    aqi_col = f'{pollutant}_individual_aqi'
    if aqi_col in aqi_results.columns:
        ax1.plot(time_vals, aqi_results[aqi_col], 
                label=pollutant.replace('_', ' ').title(), 
                alpha=0.7, linewidth=1.5, color=colors[i % len(colors)])

# Plot maximum AQI as thick black line
ax1.plot(time_vals, max_aqi_vals, 
         label='Maximum AQI (Envelope)', 
         color='black', linewidth=3, alpha=0.8)

ax1.set_ylabel('AQI Value')
ax1.set_title('Individual Pollutant AQIs vs Maximum AQI Over Time')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Add AQI level backgrounds
aqi_levels = [(0, 50, 'green'), (50, 100, 'yellow'), (100, 150, 'orange'), (150, 200, 'red')]
for min_val, max_val, color in aqi_levels:
    ax1.axhspan(min_val, max_val, alpha=0.1, color=color)

# Plot 2: Distribution of individual AQIs 
aqi_data = []
pollutant_names = []
for pollutant in available_pollutants:
    aqi_col = f'{pollutant}_individual_aqi'
    if aqi_col in aqi_results.columns:
        aqi_data.append(aqi_results[aqi_col].values)
        pollutant_names.append(pollutant.replace('_', ' ').title())

# Add maximum AQI for comparison
aqi_data.append(max_aqi_vals.values)
pollutant_names.append('Maximum AQI')

# Create box plot
bp = ax2.boxplot(aqi_data, labels=pollutant_names, patch_artist=True)

# Color the boxes
box_colors = colors + ['black']
for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax2.set_ylabel('AQI Value')
ax2.set_title('AQI Distribution by Pollutant')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Add AQI level lines
for level, color in [(50, 'green'), (100, 'yellow'), (150, 'orange')]:
    ax2.axhline(y=level, color=color, linestyle='--', alpha=0.7, linewidth=1)

plt.tight_layout()
plt.show()

# Summary statistics for contribution analysis
print("\nIndividual AQI vs Maximum AQI Analysis:")
max_aqi_mean = max_aqi_vals.mean()
print(f"Maximum AQI - Mean: {max_aqi_mean:.1f}, Range: {max_aqi_vals.min():.0f}-{max_aqi_vals.max():.0f}")

print("\nHow often each pollutant reaches within 90% of maximum AQI:")
for pollutant in available_pollutants:
    aqi_col = f'{pollutant}_individual_aqi'
    if aqi_col in aqi_results.columns:
        close_to_max = (aqi_results[aqi_col] >= 0.9 * max_aqi_vals).sum()
        percentage = (close_to_max / len(aqi_results) * 100)
        print(f"  {pollutant:<15}: {close_to_max:3d}/{len(aqi_results)} times ({percentage:5.1f}%) within 90% of max")

print("\nThis shows which pollutants are the 'runners-up' when they don't control AQI")


In [ ]:
# VISUALIZATION 3: Time Series of Controlling Pollutants
print("\n" + "="*60)
print("TIME SERIES OF AQI CONTROL")
print("="*60)

# Create a time series showing which pollutant controls AQI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: AQI comparison over time
ax1.plot(aqi_results['time'], aqi_results['us_aqi'], label='Current US AQI (PM only)', alpha=0.7, linewidth=2)
ax1.plot(aqi_results['time'], aqi_results['calculated_max_aqi'], label='True Max AQI (All pollutants)', alpha=0.7, linewidth=2)
ax1.fill_between(aqi_results['time'], aqi_results['us_aqi'], aqi_results['calculated_max_aqi'], 
                 alpha=0.3, color='red', label='Missing AQI')

ax1.set_ylabel('AQI Value')
ax1.set_title('AQI Comparison: Current PM-only vs Full EPA Calculation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add AQI level backgrounds
aqi_levels = [(0, 50, 'green'), (50, 100, 'yellow'), (100, 150, 'orange'), (150, 200, 'red')]
for min_val, max_val, color in aqi_levels:
    ax1.axhspan(min_val, max_val, alpha=0.1, color=color)

# Plot 2: Controlling pollutant over time
pollutant_colors = {
    'pm2_5': 'red', 'pm10': 'darkred', 'ozone': 'blue', 
    'carbon_monoxide': 'purple', 'nitrogen_dioxide': 'orange', 'sulphur_dioxide': 'green'
}

# Create numerical encoding for pollutants for plotting
unique_pollutants = aqi_results['controlling_pollutant'].unique()
pollutant_mapping = {p: i for i, p in enumerate(unique_pollutants)}
aqi_results['pollutant_num'] = aqi_results['controlling_pollutant'].map(pollutant_mapping)

scatter = ax2.scatter(aqi_results['time'], aqi_results['pollutant_num'], 
                     c=[pollutant_colors.get(p, 'gray') for p in aqi_results['controlling_pollutant']], 
                     alpha=0.7, s=20)

ax2.set_ylabel('Controlling Pollutant')
ax2.set_xlabel('Date')
ax2.set_title('Which Pollutant Controls AQI Over Time')
ax2.set_yticks(range(len(unique_pollutants)))
ax2.set_yticklabels([p.replace('_', ' ').title() for p in unique_pollutants])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate impact of missing other pollutants
aqi_difference = aqi_results['calculated_max_aqi'] - aqi_results['us_aqi']
significant_underestimation = (aqi_difference > 10).sum()

print(f"\n📊 IMPACT ANALYSIS:")
print(f"Times when PM-only AQI underestimates by >10 points: {significant_underestimation}/{len(aqi_results)} ({significant_underestimation/len(aqi_results)*100:.1f}%)")
print(f"Maximum underestimation: {aqi_difference.max():.1f} AQI points")
print(f"Average difference: {aqi_difference.mean():.1f} AQI points")

# Final recommendation
print(f"\n🎯 MODELING RECOMMENDATION:")
if pm_percentage >= 85:
    print("✅ PM-only approach is SUFFICIENT for Multan AQI prediction")
    print("   PM2.5 + PM10 control >85% of AQI variations")
elif pm_percentage >= 70:
    print("⚠️  PM-only approach is MOSTLY adequate but consider monitoring other pollutants")
    print("   PM2.5 + PM10 control 70-85% of AQI variations")
else:
    print("🚨 PM-only approach MISSES significant AQI drivers")
    print("   Consider including other pollutants in prediction model")

print(f"\nCurrent focus on PM2.5 and PM10 captures {pm_percentage:.1f}% of AQI control instances.")

print(f"\n🎯 FINAL ANALYSIS:")
print(f"✓ Analyzed all {len(available_pollutants)} EPA criteria pollutants in dataset")
print(f"✓ PM2.5 and PM10 are responsible for {pm_percentage:.1f}% of AQI determinations")
print(f"✓ Other criteria pollutants (O3, CO, NO2, SO2) control {100-pm_percentage:.1f}% of AQI")
print(f"✓ This validates the scope of your PM-focused modeling approach")


## 5. Lag Features Analysis

Examining the importance of historical PM values for predicting current concentrations (which leads to better AQI calculations).


In [ ]:
# Analyze lag features for BOTH ML targets (PM2.5 AND PM10)
print("="*60)
print("LAG FEATURES ANALYSIS FOR ML TARGETS")
print("="*60)

# PM10 Lag Analysis (since it's also a target)
pm10_lag_features = [col for col in df.columns if 'lag' in col and 'pm10' in col]
if pm10_lag_features:
    print(f"\nFound {len(pm10_lag_features)} PM10 lag features:")
    print(pm10_lag_features[:5], "..." if len(pm10_lag_features) > 5 else "")
    
    # Calculate correlations between current PM10 and its lag features
    pm10_lag_correlations = df[['pm10'] + pm10_lag_features].corr()['pm10'].drop('pm10')
    
    # Plot PM10 lag correlations
    plt.figure(figsize=(12, 6))
    lag_hours = [1, 2, 3, 6, 12, 24, 48, 72]
    pm10_correlations = [pm10_lag_correlations[f'pm10_lag_{h}h'] for h in lag_hours if f'pm10_lag_{h}h' in pm10_lag_correlations.index]
    
    plt.bar(range(len(pm10_correlations)), pm10_correlations, alpha=0.7, color='orange')
    plt.xlabel('Lag Hours')
    plt.ylabel('Correlation with Current PM10')
    plt.title('PM10 Lag Features Correlation (ML Target)')
    plt.xticks(range(len(pm10_correlations)), [f'{h}h' for h in lag_hours[:len(pm10_correlations)]])
    plt.grid(True, alpha=0.3)
    
    for i, corr in enumerate(pm10_correlations):
        plt.text(i, corr + 0.01, f'{corr:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\nPM10 lag feature correlations (ML Target):")
    for i, h in enumerate(lag_hours[:len(pm10_correlations)]):
        print(f"  {h:2d}h lag: {pm10_correlations[i]:.3f}")

print("\n[ML Targets: PM2.5 & PM10 concentrations]")
print("[Ultimate Goal: Accurate AQI predictions for Multan]")


In [ ]:
# Analyze lag features correlation with current PM2.5
lag_features = [col for col in df.columns if 'lag' in col and 'pm2_5' in col]
print(f"Found {len(lag_features)} PM2.5 lag features:")
print(lag_features)

if lag_features:
    # Calculate correlations between current PM2.5 and its lag features
    lag_correlations = df[['pm2_5'] + lag_features].corr()['pm2_5'].drop('pm2_5')
    
    # Plot lag correlations
    plt.figure(figsize=(12, 6))
    lag_hours = [1, 2, 3, 6, 12, 24, 48, 72]  # Expected lag hours
    correlations = [lag_correlations[f'pm2_5_lag_{h}h'] for h in lag_hours if f'pm2_5_lag_{h}h' in lag_correlations.index]
    
    plt.bar(range(len(correlations)), correlations, alpha=0.7)
    plt.xlabel('Lag Hours')
    plt.ylabel('Correlation with Current PM2.5')
    plt.title('PM2.5 Lag Features Correlation')
    plt.xticks(range(len(correlations)), [f'{h}h' for h in lag_hours[:len(correlations)]])
    plt.grid(True, alpha=0.3)
    
    for i, corr in enumerate(correlations):
        plt.text(i, corr + 0.01, f'{corr:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\nLag feature correlations with current PM2.5:")
    for i, h in enumerate(lag_hours[:len(correlations)]):
        print(f"  {h:2d}h lag: {correlations[i]:.3f}")

# Complete Rolling Statistics Analysis for Both ML Targets
print(f"\n" + "="*60)
print("ROLLING STATISTICS ANALYSIS (ML TARGETS)")
print("="*60)

# PM2.5 Rolling Features Analysis
pm25_rolling_features = [col for col in df.columns if 'rolling' in col and 'pm2_5' in col]
if pm25_rolling_features:
    print(f"\nPM2.5 ROLLING FEATURES ({len(pm25_rolling_features)} total):")
    
    # Group by statistic type
    rolling_types = ['mean', 'std', 'min', 'max']
    for stat_type in rolling_types:
        stat_features = [col for col in pm25_rolling_features if stat_type in col]
        if stat_features:
            print(f"\n  {stat_type.upper()} features:")
            correlations = df[['pm2_5'] + stat_features].corr()['pm2_5'].drop('pm2_5')
            for feature, corr in correlations.sort_values(ascending=False).items():
                window = feature.split('_')[-1]
                print(f"    {window:<4} window: {corr:.3f}")

# PM10 Rolling Features Analysis  
pm10_rolling_features = [col for col in df.columns if 'rolling' in col and 'pm10' in col]
if pm10_rolling_features:
    print(f"\nPM10 ROLLING FEATURES ({len(pm10_rolling_features)} total):")
    
    # Group by statistic type
    for stat_type in rolling_types:
        stat_features = [col for col in pm10_rolling_features if stat_type in col]
        if stat_features:
            print(f"\n  {stat_type.upper()} features:")
            correlations = df[['pm10'] + stat_features].corr()['pm10'].drop('pm10')
            for feature, corr in correlations.sort_values(ascending=False).items():
                window = feature.split('_')[-1]
                print(f"    {window:<4} window: {corr:.3f}")

# Change Rate Features Analysis
print(f"\nCHANGE RATE FEATURES:")
change_features = [col for col in df.columns if 'change_rate' in col]
for target in ['pm2_5', 'pm10']:
    target_change_features = [col for col in change_features if target in col]
    if target_change_features:
        print(f"\n  {target.upper()} change rates:")
        correlations = df[[target] + target_change_features].corr()[target].drop(target)
        for feature, corr in correlations.items():
            period = feature.split('_')[-1]
            print(f"    {period:<4} change: {corr:.3f}")

print(f"\n🎯 KEY INSIGHTS:")
print(f"• Rolling features capture trend information over different time windows")
print(f"• Shorter windows (3h, 6h) typically correlate more strongly with current values")
print(f"• Change rates show how much PM concentrations are shifting")
print(f"• These features help models understand pollution persistence and trends")


## 6. Data Quality Analysis

**Focus**: Ensuring data reliability for accurate PM concentration predictions and AQI calculations.

### 6.1 Missing Values & Null Analysis

In [ ]:
# 6.1 Missing Values & Null Analysis
print("=" * 60)
print("MISSING VALUES & NULL ANALYSIS")
print("=" * 60)

# Basic missing value check
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percent
}).round(2)

print("EXPLICIT NULL VALUES:")
if missing_data.sum() == 0:
    print("✓ No explicit null values found!")
else:
    print("Missing values found:")
    for col in missing_df[missing_df['Missing_Count'] > 0].index:
        count = missing_df.loc[col, 'Missing_Count']
        percent = missing_df.loc[col, 'Missing_Percentage']
        print(f"  {col:<25} {count:>6} ({percent:>6.2f}%)")

# Check for zero values that might represent missing data
print(f"\nZERO VALUES ANALYSIS (Potential Missing Data):")
air_quality_cols = ['pm2_5', 'pm10', 'carbon_monoxide', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide']
zero_issues = {}

for col in air_quality_cols:
    if col in df.columns:
        zero_count = (df[col] == 0.0).sum()
        zero_percent = (zero_count / len(df)) * 100
        if zero_count > 0:
            zero_issues[col] = {'count': zero_count, 'percent': zero_percent}

if zero_issues:
    print("⚠️  ZERO VALUES FOUND (may indicate missing sensors):")
    for col, stats in zero_issues.items():
        print(f"  {col:<20}: {stats['count']:>3d} zeros ({stats['percent']:>5.1f}%)")
        
    # Visualize zero patterns over time
    fig, axes = plt.subplots(len(zero_issues), 1, figsize=(15, 3*len(zero_issues)))
    if len(zero_issues) == 1:
        axes = [axes]
    
    for i, (col, stats) in enumerate(zero_issues.items()):
        zero_mask = df[col] == 0.0
        axes[i].scatter(df[zero_mask]['time'], [col]*zero_mask.sum(), 
                       alpha=0.7, color='red', s=20, label=f'Zero values ({stats["count"]})')
        axes[i].set_ylabel(col.replace('_', ' ').title())
        axes[i].set_title(f'{col.replace("_", " ").title()} - Zero Value Timeline')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
        
    plt.xlabel('Date')
    plt.tight_layout()
    plt.show()
else:
    print("✓ No zero values in air quality parameters")

# Check for consecutive missing periods (data gaps)
print(f"\nTIME GAP ANALYSIS:")
df_sorted = df.sort_values('time').reset_index(drop=True)
time_diffs = df_sorted['time'].diff()
large_gaps = time_diffs[time_diffs > pd.Timedelta(hours=2)]

if len(large_gaps) > 0:
    print(f"⚠️  Found {len(large_gaps)} time gaps > 2 hours:")
    for idx, gap in large_gaps.items():
        gap_start = df_sorted.loc[idx-1, 'time'] if idx > 0 else 'Start'
        gap_end = df_sorted.loc[idx, 'time']
        print(f"  Gap: {gap} between {gap_start} and {gap_end}")
else:
    print("✓ No significant time gaps found")


### 6.2 Data Consistency & Physics Validation


In [ ]:
# 6.2 Data Consistency & Physics Validation
print("=" * 60)
print("DATA CONSISTENCY & PHYSICS VALIDATION")
print("=" * 60)

# Check PM2.5 vs PM10 relationship (PM2.5 should generally be ≤ PM10)
if 'pm2_5' in df.columns and 'pm10' in df.columns:
    pm_violations = df[df['pm2_5'] > df['pm10']]
    violation_count = len(pm_violations)
    violation_percent = (violation_count / len(df)) * 100
    
    print(f"PM2.5 vs PM10 CONSISTENCY:")
    if violation_count > 0:
        print(f"⚠️  {violation_count} records ({violation_percent:.1f}%) where PM2.5 > PM10")
        print(f"   Max violation: PM2.5={pm_violations['pm2_5'].max():.1f} > PM10={pm_violations['pm10'].max():.1f}")
        
        # Show violation timeline
        plt.figure(figsize=(15, 6))
        plt.scatter(df['time'], df['pm2_5'], alpha=0.5, label='PM2.5', s=10)
        plt.scatter(df['time'], df['pm10'], alpha=0.5, label='PM10', s=10)
        plt.scatter(pm_violations['time'], pm_violations['pm2_5'], 
                   color='red', label=f'PM2.5 > PM10 ({violation_count} cases)', s=30, marker='x')
        plt.xlabel('Date')
        plt.ylabel('Concentration (µg/m³)')
        plt.title('PM2.5 vs PM10 Consistency Check')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("✓ PM2.5 ≤ PM10 in all records (physically consistent)")

# Check reasonable ranges for weather parameters
print(f"\nWEATHER PARAMETER VALIDATION:")
weather_ranges = {
    'temperature': (-50, 60, '°C'),  # Extreme but possible range
    'humidity': (0, 100, '%'),       # Should be 0-100%
    'pressure': (800, 1200, 'hPa'),  # Reasonable atmospheric pressure
    'wind_speed': (0, 50, 'm/s'),    # Reasonable wind speeds
    'wind_direction': (0, 360, '°')  # Should be 0-360 degrees
}

weather_violations = {}
for param, (min_val, max_val, unit) in weather_ranges.items():
    if param in df.columns:
        below_min = (df[param] < min_val).sum()
        above_max = (df[param] > max_val).sum()
        violations = below_min + above_max
        
        if violations > 0:
            weather_violations[param] = {
                'below_min': below_min, 'above_max': above_max, 
                'min_val': min_val, 'max_val': max_val, 'unit': unit,
                'actual_min': df[param].min(), 'actual_max': df[param].max()
            }

if weather_violations:
    print("⚠️  WEATHER PARAMETER VIOLATIONS:")
    for param, stats in weather_violations.items():
        print(f"  {param:<15}: {stats['below_min']} below {stats['min_val']}{stats['unit']}, "
              f"{stats['above_max']} above {stats['max_val']}{stats['unit']}")
        print(f"                   Actual range: {stats['actual_min']:.1f} - {stats['actual_max']:.1f}{stats['unit']}")
else:
    print("✓ All weather parameters within reasonable ranges")

# Check cyclical feature ranges (sin/cos should be in [-1, 1])
print(f"\nCYCLICAL FEATURE VALIDATION:")
cyclical_cols = [col for col in df.columns if '_sin' in col or '_cos' in col]
cyclical_violations = {}

for col in cyclical_cols:
    below_neg1 = (df[col] < -1.01).sum()  # Small tolerance for floating point
    above_pos1 = (df[col] > 1.01).sum()
    violations = below_neg1 + above_pos1
    
    if violations > 0:
        cyclical_violations[col] = {
            'below_neg1': below_neg1, 'above_pos1': above_pos1,
            'actual_min': df[col].min(), 'actual_max': df[col].max()
        }

if cyclical_violations:
    print("⚠️  CYCLICAL FEATURE VIOLATIONS:")
    for col, stats in cyclical_violations.items():
        print(f"  {col:<20}: {stats['below_neg1']} below -1, {stats['above_pos1']} above 1")
        print(f"                       Actual range: {stats['actual_min']:.6f} - {stats['actual_max']:.6f}")
else:
    print("✓ All cyclical features within [-1, 1] range")

# Check binary flag consistency
print(f"\nBINARY FLAG VALIDATION:")
binary_cols = [col for col in df.columns if col.startswith('is_')]
binary_violations = {}

for col in binary_cols:
    unique_vals = df[col].unique()
    expected_vals = {0.0, 1.0}
    unexpected_vals = set(unique_vals) - expected_vals
    
    if unexpected_vals:
        binary_violations[col] = {
            'unexpected': list(unexpected_vals),
            'unique_vals': list(unique_vals)
        }

if binary_violations:
    print("⚠️  BINARY FLAG VIOLATIONS:")
    for col, stats in binary_violations.items():
        print(f"  {col:<20}: Found {stats['unexpected']} (expected only 0.0, 1.0)")
        print(f"                       All values: {stats['unique_vals']}")
else:
    print("✓ All binary flags contain only 0.0 and 1.0 values")


### 6.3 Statistical Outlier Detection

In [ ]:
# 6.3 Statistical Outlier Detection
print("=" * 60)
print("STATISTICAL OUTLIER DETECTION")
print("=" * 60)

# Focus on ML targets and key environmental features
key_features_for_outliers = [
    'pm2_5', 'pm10',  # ML targets - critical for model quality
    'temperature', 'humidity', 'pressure', 'wind_speed',  # Environmental predictors
    'carbon_monoxide', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide'  # Air quality predictors
]

available_outlier_features = [col for col in key_features_for_outliers if col in df.columns]

# Z-score method (|z| > 3 considered outlier)
print("Z-SCORE OUTLIER DETECTION (|z-score| > 3):")
zscore_outliers = {}

for col in available_outlier_features:
    z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
    outlier_mask = z_scores > 3
    outlier_count = outlier_mask.sum()
    
    if outlier_count > 0:
        zscore_outliers[col] = {
            'count': outlier_count,
            'percent': (outlier_count / len(df)) * 100,
            'max_zscore': z_scores.max(),
            'outlier_values': df.loc[outlier_mask, col].tolist()
        }

if zscore_outliers:
    print("⚠️  Z-SCORE OUTLIERS FOUND:")
    for col, stats in zscore_outliers.items():
        print(f"  {col:<20}: {stats['count']} outliers ({stats['percent']:.1f}%) - Max |z|: {stats['max_zscore']:.2f}")
        if col in ['pm2_5', 'pm10']:  # Show details for ML targets
            print(f"                       Values: {sorted(stats['outlier_values'])[:5]}..." if len(stats['outlier_values']) > 5 else f"                       Values: {sorted(stats['outlier_values'])}")
else:
    print("✓ No z-score outliers found (|z| > 3)")

# IQR method (values beyond Q1 - 1.5*IQR or Q3 + 1.5*IQR)
print(f"\nIQR OUTLIER DETECTION (beyond Q1-1.5*IQR, Q3+1.5*IQR):")
iqr_outliers = {}

for col in available_outlier_features:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    outlier_count = outlier_mask.sum()
    
    if outlier_count > 0:
        iqr_outliers[col] = {
            'count': outlier_count,
            'percent': (outlier_count / len(df)) * 100,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'outlier_values': df.loc[outlier_mask, col].tolist()
        }

if iqr_outliers:
    print("⚠️  IQR OUTLIERS FOUND:")
    for col, stats in iqr_outliers.items():
        print(f"  {col:<20}: {stats['count']} outliers ({stats['percent']:.1f}%)")
        print(f"                       Expected range: {stats['lower_bound']:.2f} - {stats['upper_bound']:.2f}")
        if col in ['pm2_5', 'pm10']:  # Show details for ML targets
            extreme_values = sorted(stats['outlier_values'])
            display_values = extreme_values[:3] + ['...'] + extreme_values[-2:] if len(extreme_values) > 5 else extreme_values
            print(f"                       Outlier values: {display_values}")
else:
    print("✓ No IQR outliers found")

# Visualize outliers for ML targets
ml_targets = ['pm2_5', 'pm10']
ml_targets_present = [col for col in ml_targets if col in df.columns]

if ml_targets_present and (zscore_outliers or iqr_outliers):
    print(f"\nOUTLIER VISUALIZATION FOR ML TARGETS:")
    
    fig, axes = plt.subplots(len(ml_targets_present), 2, figsize=(15, 5*len(ml_targets_present)))
    if len(ml_targets_present) == 1:
        axes = axes.reshape(1, -1)
    
    for i, target in enumerate(ml_targets_present):
        # Box plot
        ax1 = axes[i, 0]
        bp = ax1.boxplot([df[target].dropna()], patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        ax1.set_title(f'{target.upper()} - Box Plot (IQR Outliers)')
        ax1.set_ylabel(f'{target.replace("_", ".")} (µg/m³)')
        ax1.grid(True, alpha=0.3)
        
        # Time series with outliers highlighted
        ax2 = axes[i, 1]
        ax2.plot(df['time'], df[target], alpha=0.7, linewidth=1, label=target.upper())
        
        # Highlight outliers
        if target in zscore_outliers:
            z_scores = np.abs((df[target] - df[target].mean()) / df[target].std())
            zscore_mask = z_scores > 3
            ax2.scatter(df[zscore_mask]['time'], df.loc[zscore_mask, target], 
                       color='red', s=30, marker='x', label=f'Z-score outliers ({zscore_outliers[target]["count"]})')
        
        if target in iqr_outliers:
            Q1 = df[target].quantile(0.25)
            Q3 = df[target].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            iqr_mask = (df[target] < lower_bound) | (df[target] > upper_bound)
            ax2.scatter(df[iqr_mask]['time'], df.loc[iqr_mask, target], 
                       color='orange', s=20, marker='o', alpha=0.7, label=f'IQR outliers ({iqr_outliers[target]["count"]})')
        
        ax2.set_title(f'{target.upper()} - Time Series with Outliers')
        ax2.set_xlabel('Date')
        ax2.set_ylabel(f'{target.replace("_", ".")} (µg/m³)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Summary of outlier impact on ML targets
print(f"\n🎯 OUTLIER IMPACT ON ML TARGETS:")
for target in ml_targets_present:
    total_outliers = 0
    if target in zscore_outliers:
        total_outliers += zscore_outliers[target]['count']
    if target in iqr_outliers:
        total_outliers += iqr_outliers[target]['count']  # Note: may overlap with z-score
    
    if total_outliers > 0:
        outlier_percent = (total_outliers / len(df)) * 100
        print(f"  {target.upper()}: ~{total_outliers} potential outliers ({outlier_percent:.1f}% of data)")
        print(f"           Consider: Review for sensor errors vs genuine extreme pollution events")
    else:
        print(f"  {target.upper()}: No significant outliers detected")

print(f"\n[Quality Note: Outliers in PM concentrations could be genuine pollution spikes or sensor malfunctions]")
print(f"[Recommendation: Investigate extreme values before removal - they might be real air quality events]")


### 6.4 Data Completeness & Summary Quality Report

In [ ]:
# 6.4 Data Completeness & Summary Quality Report
print("=" * 60)
print("DATA COMPLETENESS & SUMMARY QUALITY REPORT")
print("=" * 60)

# Duplicate detection
duplicates = df.duplicated().sum()
print(f"DUPLICATE RECORDS: {duplicates}")
if duplicates > 0:
    print("⚠️  Duplicate rows found - consider deduplication")
else:
    print("✓ No duplicate rows")

# Unique values analysis
print(f"\nUNIQUE VALUES ANALYSIS:")
feature_categories = {
    'ML Targets': ['pm2_5', 'pm10'],
    'Air Quality': ['carbon_monoxide', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide', 'nh3'],
    'AQI Calculations': ['pm2_5_aqi', 'pm10_aqi', 'us_aqi', 'openweather_aqi'],
    'Weather': ['temperature', 'humidity', 'pressure', 'wind_speed', 'wind_direction'],
    'Time Features': ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'],
    'Binary Flags': [col for col in df.columns if col.startswith('is_')],
    'Lag Features': [col for col in df.columns if 'lag' in col][:5],  # Show first 5
    'Rolling Features': [col for col in df.columns if 'rolling' in col][:5]  # Show first 5
}

for category, cols in feature_categories.items():
    available_cols = [col for col in cols if col in df.columns]
    if available_cols:
        print(f"\n{category}:")
        for col in available_cols:
            unique_count = df[col].nunique()
            unique_ratio = unique_count / len(df)
            if unique_ratio < 0.01:  # Very low uniqueness
                print(f"  {col:<25} {unique_count:>6} unique ({unique_ratio:>6.2%}) ⚠️  Low variation")
            elif unique_ratio > 0.95:  # Very high uniqueness  
                print(f"  {col:<25} {unique_count:>6} unique ({unique_ratio:>6.2%}) ✓ High variation")
            else:
                print(f"  {col:<25} {unique_count:>6} unique ({unique_ratio:>6.2%})")

# Data quality scoring
print(f"\n" + "=" * 60)
print("OVERALL DATA QUALITY SCORE")
print("=" * 60)

quality_score = 100  # Start with perfect score
quality_issues = []

# Deduct points for various issues
if missing_data.sum() > 0:
    missing_percent_total = (missing_data.sum() / (len(df) * len(df.columns))) * 100
    deduction = min(20, missing_percent_total * 4)  # Up to 20 points for missing data
    quality_score -= deduction
    quality_issues.append(f"Missing data: -{deduction:.1f} points ({missing_percent_total:.1f}% of all values)")

if duplicates > 0:
    dup_percent = (duplicates / len(df)) * 100
    deduction = min(10, dup_percent * 2)  # Up to 10 points for duplicates
    quality_score -= deduction
    quality_issues.append(f"Duplicate rows: -{deduction:.1f} points ({dup_percent:.1f}% of records)")

if weather_violations:
    deduction = len(weather_violations) * 2  # 2 points per violated weather parameter
    quality_score -= deduction
    quality_issues.append(f"Weather violations: -{deduction} points ({len(weather_violations)} parameters)")

if cyclical_violations:
    deduction = len(cyclical_violations) * 3  # 3 points per violated cyclical feature
    quality_score -= deduction
    quality_issues.append(f"Cyclical violations: -{deduction} points ({len(cyclical_violations)} features)")

if binary_violations:
    deduction = len(binary_violations) * 2  # 2 points per violated binary flag
    quality_score -= deduction
    quality_issues.append(f"Binary flag violations: -{deduction} points ({len(binary_violations)} flags)")

# Check PM2.5 vs PM10 physics violations
if 'pm2_5' in df.columns and 'pm10' in df.columns:
    pm_violations = (df['pm2_5'] > df['pm10']).sum()
    if pm_violations > 0:
        violation_percent = (pm_violations / len(df)) * 100
        deduction = min(15, violation_percent * 3)  # Up to 15 points for physics violations
        quality_score -= deduction
        quality_issues.append(f"PM physics violations: -{deduction:.1f} points ({violation_percent:.1f}% of records)")

# Outlier penalty (mild)
total_outlier_features = len(zscore_outliers) + len(iqr_outliers)
if total_outlier_features > 0:
    deduction = min(5, total_outlier_features * 0.5)  # Mild penalty for outliers
    quality_score -= deduction
    quality_issues.append(f"Statistical outliers: -{deduction:.1f} points (in {total_outlier_features} features)")

# Ensure score doesn't go below 0
quality_score = max(0, quality_score)

# Display results
print(f"📊 DATA QUALITY SCORE: {quality_score:.1f}/100")

if quality_score >= 90:
    status = "🟢 EXCELLENT"
    recommendation = "Data is ready for high-quality ML modeling"
elif quality_score >= 75:
    status = "🟡 GOOD" 
    recommendation = "Data is suitable for ML with minor preprocessing"
elif quality_score >= 60:
    status = "🟠 ACCEPTABLE"
    recommendation = "Address major issues before ML modeling"
else:
    status = "🔴 POOR"
    recommendation = "Significant data cleaning required"

print(f"Status: {status}")
print(f"Recommendation: {recommendation}")

if quality_issues:
    print(f"\nISSUES IDENTIFIED:")
    for issue in quality_issues:
        print(f"  • {issue}")
else:
    print(f"\n✅ NO SIGNIFICANT QUALITY ISSUES DETECTED")

print(f"\n🎯 ML READINESS ASSESSMENT:")
ml_targets = ['pm2_5', 'pm10']
ml_readiness = True

for target in ml_targets:
    if target in df.columns:
        target_quality = 100
        target_issues = []
        
        # Check for issues specific to ML targets
        if target in zscore_outliers:
            outlier_percent = zscore_outliers[target]['percent']
            if outlier_percent > 5:
                target_quality -= 10
                target_issues.append(f"High outlier rate ({outlier_percent:.1f}%)")
        
        zero_count = (df[target] == 0.0).sum()
        if zero_count > len(df) * 0.1:  # More than 10% zeros
            target_quality -= 15
            target_issues.append(f"Many zero values ({zero_count} records)")
        
        if target_issues:
            print(f"  {target.upper()}: {target_quality}/100 - {', '.join(target_issues)}")
            if target_quality < 70:
                ml_readiness = False
        else:
            print(f"  {target.upper()}: ✅ Ready for ML modeling")

if ml_readiness:
    print(f"\n✅ DATASET IS READY FOR ML MODEL TRAINING")
else:
    print(f"\n⚠️  CONSIDER ADDITIONAL PREPROCESSING FOR OPTIMAL ML PERFORMANCE")

print(f"\n[Next Steps: Feature engineering validation, train/test splitting, model selection]")


In [ ]:
# Data validation checks
print("=" * 50)
print("DATA VALIDATION CHECKS")
print("=" * 50)

# Check for negative values in air quality parameters (shouldn't be negative)
negative_check_cols = ['pm2_5', 'pm10', 'co', 'no2', 'so2', 'o3', 'us_aqi', 'pm2_5_aqi', 'pm10_aqi']
negative_issues = {}

for col in negative_check_cols:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        if negative_count > 0:
            negative_issues[col] = negative_count

if negative_issues:
    print("⚠️  NEGATIVE VALUES FOUND (unexpected):")
    for col, count in negative_issues.items():
        print(f"  {col}: {count} negative values")
else:
    print("✓ No unexpected negative values in air quality parameters")

# Check for infinite values
infinite_issues = {}
for col in numeric_cols:
    if col in df.columns:
        infinite_count = np.isinf(df[col]).sum()
        if infinite_count > 0:
            infinite_issues[col] = infinite_count

if infinite_issues:
    print("\n⚠️  INFINITE VALUES FOUND:")
    for col, count in infinite_issues.items():
        print(f"  {col}: {count} infinite values")
else:
    print("\n✓ No infinite values found")

# Check timestamp continuity
print(f"\nTIMESTAMP ANALYSIS:")
if 'time' in df.columns:
    df_sorted = df.sort_values('time')
    time_diffs = df_sorted['time'].diff().dropna()
    
    print(f"  Earliest record: {df['time'].min()}")
    print(f"  Latest record: {df['time'].max()}")
    print(f"  Total duration: {(df['time'].max() - df['time'].min()).days} days")
    print(f"  Most common interval: {time_diffs.mode().iloc[0] if len(time_diffs.mode()) > 0 else 'N/A'}")
    print(f"  Records per day average: {len(df) / max(1, (df['time'].max() - df['time'].min()).days):.1f}")
